# Tensor

In [1]:
import tensorflow as tf

In [2]:
# zero rank tensor
cat = tf.Variable("Jerry", tf.string)
age = tf.Variable(9, tf.int8)
pai = tf.Variable(3.14159265359, tf.float64)
complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

[cat, age, pai, complicated]

[<tf.Variable 'Variable:0' shape=() dtype=string_ref>,
 <tf.Variable 'Variable_1:0' shape=() dtype=int32_ref>,
 <tf.Variable 'Variable_2:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=() dtype=complex128_ref>]

In [3]:
# one rank tensor
one_str = tf.Variable(["hello"], tf.string)
one_float = tf.Variable([3.1224, 2.445], tf.float16)
one_int = tf.Variable([2, 4, 6, 7], tf.int16)
one_complicated = tf.Variable([13.2 - 4.23j, 34.1 - 86.4j], tf.complex64)

[one_str, one_float, one_int, one_complicated]

[<tf.Variable 'Variable_4:0' shape=(1,) dtype=string_ref>,
 <tf.Variable 'Variable_5:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_6:0' shape=(4,) dtype=int32_ref>,
 <tf.Variable 'Variable_7:0' shape=(2,) dtype=complex128_ref>]

In [4]:
# two rank tensor
two_int = tf.Variable([[7], [12]], tf.int16)
two_bool = tf.Variable([[False, True], [True, False]], tf.bool)
two_int32 = tf.Variable([[3, 2], [5, 7], [73, 2]], tf.int32)
rank32 = tf.rank(two_int32)

[two_int, two_bool, two_int32, rank32]

[<tf.Variable 'Variable_8:0' shape=(2, 1) dtype=int32_ref>,
 <tf.Variable 'Variable_9:0' shape=(2, 2) dtype=bool_ref>,
 <tf.Variable 'Variable_10:0' shape=(3, 2) dtype=int32_ref>,
 <tf.Tensor 'Rank:0' shape=() dtype=int32>]

In [6]:
# four rank tensor, defined a tensor with zeros method.
f = tf.zeros([10, 28, 273, 34])

# tensor will be recycled automatic, variable is not.
f

<tf.Tensor 'zeros_1:0' shape=(10, 28, 273, 34) dtype=float32>

# Variable

variable is a special tensor

- 张量的生命周期通常随依赖的计算完成而结束，内存也随即释放。
- 变量则常驻内存，在每一步训练时不断更新值，以实现模型参数的更新。

## Create variables

```
# create var
w = tf.Variable(<initial-value>, name=<optional-name>)

# put vars as input
y = tf.matmul(w, ...another variable or tensor...)
z = tf.sigmoid(w + y)

# set value for vars with assign or assign_xxx methods
w.assign(w + 1.0)
w.assign_add(1.0)
```

In [2]:
# tf.random_normal 方法返回形状为（1，4）的张量。它的4个元素符合均值为100，标准差为0.35的正态分布。
w = tf.Variable(initial_value=tf.random_normal(shape=(1, 4), mean=100, stddev=0.35), name="w")
b = tf.Variable(tf.zeros([4]), name="b")

In [5]:
[w, b]

[<tf.Variable 'w:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'b:0' shape=(4,) dtype=float32_ref>]

In [4]:
# initialize variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run([w, b])

[array([[100.04436 , 100.053345, 100.131775, 100.08898 ]], dtype=float32),
 array([0., 0., 0., 0.], dtype=float32)]

In [6]:
# update variable b and show update operator status. operator success is not equals b is updated successful.
sess.run(tf.assign_add(b, [1, 1, 1, 1]))

array([1., 1., 1., 1.], dtype=float32)

In [7]:
# check b is updated success.
sess.run(b)

array([1., 1., 1., 1.], dtype=float32)

## Saver demo

```
v1 = tf.Variable(..., name='v1')
v2 = tf.Variable(..., name='v2')

# save variables
saver = tf.train.Saver({'v1':v1, 'v2':v2})
saver = tf.train.Saver([v1, v2])
saver = tf.train.Saver({v.op.name: v for v in [v1, v2]})

# call saver method
tf.train.saver.save(sess, 'file-name', global_step=0)
```

In [9]:
# create saver
saver = tf.train.Saver({'w': w, 'b': b})
# save
saver.save(sess, './saver/test-ckpt', global_step=0)

'./saver/test-ckpt-0'

In [10]:
# updated b
sess.run(tf.assign_add(b, [1, 1, 1, 1]))
sess.run(b)

array([2., 2., 2., 2.], dtype=float32)

In [11]:
# reload b from checkpoint files
saver.restore(sess, './saver/test-ckpt-0')
sess.run(b)

INFO:tensorflow:Restoring parameters from ./saver/test-ckpt-0


array([1., 1., 1., 1.], dtype=float32)

**从 checkpoint 文件中恢复数据流图结构**
```
tf.train.import_meta_graph
```

# Operator

TensorFlow 用数据流图表示算法模型。数据流图由节点和有向边组成，每个节点均对应一个具体的操作。因此，操作是模型功能的$\color{#ea4339}{实际}\color{#ea4339}{载体}$。

数据流图中的节点按照功能不同可以分为3种：

- 存储节点：有状态的变量操作，通常用来存储模型参数；
- 计算节点：无状态的计算或控制操作，主要负责算法逻辑表达或流程控制；
- 数据节点：数据的占位符操作，用于描述图外输入数据的属性；

$\color{#ea4339}{操作的输入和输出是张量或操作（函数式编程）}$

<img width=80% height=80% src="imgs/03/01.jpeg" alt="imgs/03/01.jpeg" title="图1" />

TensorFlow使用占位符操作表示图外输入的数据，如训练和测试数据。

TensorFlow数据流图描述流算法模型的计算拓扑，其中的各个操作（节点）都是抽象的函数映射或数学表达式。

换句话说，数据流图本身是一个具有计算拓扑和内部结构的“壳”。

用户向数据流图填充数据前，图中并没有真正执行任何计算。

<img width=80% height=80% src="imgs/03/02.jpeg" alt="imgs/03/02.jpeg" title="图2" />

In [12]:
import tensorflow as tf

In [13]:
# constant operator
a = tf.constant(2)
b = tf.constant(3)

In [14]:
with tf.Session() as sess:
    print("a: %i" % sess.run(a))
    print("b: %i" % sess.run(b))
    print("Addition with constants: %i" % sess.run(a + b))
    print("Multiplication with constants: %i" % sess.run(a * b))

a: 2
b: 3
Addition with constants: 5
Multiplication with constants: 6


In [15]:
# placeholder operator
x = tf.placeholder(tf.int16, shape=(), name="x")
y = tf.placeholder(tf.int16, shape=(), name="y")

In [17]:
add = tf.add(x, y)
mul = tf.multiply(x, y)

In [18]:
# errors for not feed a value for placeholder tensor
with tf.Session() as sess:
    print("Addition with variables: %i" % sess.run(add))
    print("Multiplication with variables: %i" % sess.run(mul))

InvalidArgumentError: You must feed a value for placeholder tensor 'y' with dtype int16
	 [[node y (defined at <ipython-input-15-4eb7d8b6e8dd>:3)  = Placeholder[dtype=DT_INT16, shape=[], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op u'y', defined at:
  File "/usr/local/Cellar/python@2/2.7.17/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/Cellar/python@2/2.7.17/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/po/Documents/repo/venv/py2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/po/Documents/repo/venv/py2/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/po/Documents/repo/venv/py2/lib/python2.7/site-packages/ipykernel/kernel

In [ ]:
# feed a value and running success
with tf.Session() as sess:
    print("Addition with variables: %i" % sess.run(add, feed_dict={x: 10, y: 7}))
    print("Multiplication with variables: %i" % sess.run(mul, feed_dict={x: 3, y: 9}))

# Session

会话提供了估算张量和执行操作的运行环境，它是发放计算任务的客户端，所有计算任务都有它连接的执行引擎完成。一个会话的典型使用流程分为以下3步：

```
# create session
sess = tf.Session(target=..., graph=..., config=...)
# run operation
sess.run(...)
# close session
sess.close()
```

| 参数名称 | 功能说明 |
| :------: | :------: |
| target | 会话连接的执行引擎 |
| graph | 会话加载的数据流图 |
| config | 会话启动时的配置项 |

<img width=80% height=80% src="imgs/03/03.jpeg" alt="imgs/03/03.jpeg" title="图3" />

In [23]:
import tensorflow as tf

x = tf.placeholder(tf.float32)
W = tf.Variable(1.0)
b = tf.Variable(1.0)
y = W * x + b

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    fetch = y.eval(feed_dict={x: 3.0})
    print(">>> " + str(fetch))

>>> 4.0


## TensorFlow会话执行原理

当我们调用sess.run(train_op)语句执行训练操作时：

- 首先，程序内部提取操作依赖的所有前置操作。这些操作的节点共同组成一副子图。
- 然后，程序会将子图中的计算节点、存储节点和数据节点按照各自的执行设备分类，相同设备上的节点组成了一副局部图。
- 最后，每个设备上的局部图在实际执行时，根据节点间的依赖关系将各个节点有序地加载到设备上执行。

对于单机程序来说，相同机器上不同编号的CPU或GPU就是不同的设备，我们可以在创建节点时指定执行该节点的设备。

```
# run on cpu:0 device
with tf.device("/cpu:0"):
    v = tf.Variable(...)
    
# run on gpu:0 device
with tf.device("/gpu:0"):
    z = tf.matmul(x, y)
```

TensorFlow本地计算的简单示例，如下图：

Client是pythonAPI写好的数据流图，但数据流图并不是在python的解释器里执行的，需要连接远程的一个执行引擎，也就是说session连接是Server对应的底层C++层的核心引擎。

<img width=80% height=80% src="imgs/03/04.jpeg" alt="imgs/03/04.jpeg" title="图4" />

当执行sess.run()时，数据流图的各个部分子图中的节点根据根据绑定的设备关系被划分到不同的设备中进行计算，然后根据拓扑图执行入度为0的节点。

<img width=80% height=80% src="imgs/03/05.jpeg" alt="imgs/03/05.jpeg" title="图5" />

# Optimizer

## 前置知识：损失函数

损失函数是评估特定模型参数和特定输入时，表达模型输出的推理值与真实值之间不一致程度的函数。损失函数L的形势化定义如下：

$loss = L(f(x_i; \theta), y_{i})$

常见的损失函数有平方损失函数、交叉熵损失函数和指数损失函数：

$loss = (y_i-f(x_i;\theta))^2$

$loss = y_i*log(f(x_i;\theta))$

$loss = exp(-y_i*f(x_i;\theta))$

使用损失函数对所有训练样本求损失值，再累加求平均可得到模型的经验风险。换句话说，$f(x)$关于训练集的平均损失就是经验风险，其形式化定义如下：

$R_{emp}(f) = \frac{1}{N} \sum_{i=1}^{N} L(f(x_i;\theta),y_i)$

然而，如果过度的追求训练数据上的低损失值，就会遇到过拟合问题。训练集通常并不能完全代表真实场景的数据分布。当两者的分布不一致时，如果过分依赖训练集上的数据，面对新数据时就会无所适从，这时模型的$\color{#ea4339}{泛化能力就会变差}$。

模型训练的目标是不断最小化经验风险。随着训练步数的增加，经验风险将逐渐降低，模型复杂度也将逐渐上升。为了降低过度训练可能造成的过度拟合风险，可以引入专门用来度量模型复杂度的正则化项（regularizer）或惩罚项（penalty term）-J(f)。常用的正则化项有L0、L1和L2范数。因此，我们将模型最优化的目标替换为$\color{#ea4339}{鲁棒性更好}$的结构风险最小化（structural risk minimization, SRM）。如下所示，它由经验风险项和正则项两部分构成：

$R_{srm}(f) = min\frac{1}{N}\sum_{i=1}^{N}L(f(x_i;\theta),y_i)+\lambda J(\theta)$

在模型训练过程中，结构风险不断的降低。当小于我们设置的损失值阈值时，则认为此时的模型已经满足需求。因此，模型训练的本质就是在最小化结构风险的同时取得最优的模型参数。

最优模型参数的形式化定义如下：

$\theta^* = arg \min \limits_{\theta} R_{srm}(f) = arg \min \limits_{\theta} \frac{1}{N} \sum_{i=1}^{N} L(f(x_i;\theta),y_i) + \lambda J(\theta)$

## 前置知识：优化算法

典型的机器学习和深度学习问题通常都需要转换为$\color{#ea4339}{最优化问题}$进行求解。

求解最优化问题的算法称为$\color{#ea4339}{优化算法}$，他们通常采用$\color{#ea4339}{迭代方式}$实现：首先设定一个初始的可行解，然后基于特定的函数反复重新计算可行解，直到找到一个最优解或达到预设的收敛条件。不同的优化算法采用的迭代策略各有不同：

- 有的使用目标函数的一阶导数，如梯度下降法；

- 有的使用目标函数的二阶导数，如牛顿法；

- 有的使用前几轮迭代的信息，如Adam。

基于梯度下降法的迭代策略最简单，它直接沿着梯度负方向，即$\color{#ea4339}{目标函数减小最快}$的方向进行直线搜索。其计算表达式如下：

$x_{k+1} = x_k - a*grad(x_k)$

## TensorFlow训练机制

典型的机器学习和深度学习问题，包含以下3个部分：

- 1.$\color{#ea4339}{模型}$：$y=f(x)=wx+b$，其中$x$是输入数据，$y$是模型输出的推理值，$w$和$b$是模型参数，即用户的训练对象。

- 2.$\color{#ea4339}{损失函数}$：$loss=L(y,y_i)$,其中$y_i$是$x$对应的真实值（标签），$loss$为损失函数输出的损失值。

- 3.$\color{#ea4339}{优化算法}$：$w\leftarrow w+a*grad(w)$，$b\leftarrow b+a*grad(b)$，其中$grad(w)$和$grad(b)$分别表示当损失值为$loss$时，模型参数$w$和$b$各自的梯度值，$a$为学习率。

## TensorFlow优化器

$\color{#ea4339}{优化器}$是实现优化算法的载体。一次典型的迭代优化应该分为以下3个步骤：

- 1.$\color{#ea4339}{计算梯度}$：调用compute_gradients方法；

- 2.$\color{#ea4339}{处理梯度}$：用户按照自己需求处理梯度值，如梯度裁剪和梯度加权等；

- 3.$\color{#ea4339}{应用梯度}$：调用apply_gradients方法，将处理后的梯度值应用到模型参数。

<img width=80% height=80% src="imgs/03/06.jpeg" alt="imgs/03/06.jpeg" title="图6" />

TensorFlow优化器训练过程：

<img width=80% height=80% src="imgs/03/07.jpeg" alt="imgs/03/07.jpeg" title="图7" />

<img width=80% height=80% src="imgs/03/08.jpeg" alt="imgs/03/08.jpeg" title="图8" />

TensorFlow内置优化器：

<img width=80% height=80% src="imgs/03/09.jpeg" alt="imgs/03/09.jpeg" title="图9" />